# Constructing a Grid from Points



In [ ]:
import uxarray as ux
import xarray as xr
import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import warnings

warnings.filterwarnings("ignore")

## Data


| Distribution           | Example                          |
|------------------------|----------------------------------|
| Global Data            | Global Atmosphere Mesh          |
| Global Data with Holes | Global Ocean Mesh               |
| Regional Data          | Geostationary point observations |
| Regional Data with Holes        | TODO: |


In this notebook, we will be using data from 

In [ ]:
uxgrid_global = ux.open_grid("../../test/meshfiles/ugrid/outCSne30/outCSne30.ug")
uxgrid_global_ocean = ux.open_grid("../../test/meshfiles/mpas/QU/oQU480.231010.nc")
uxgrid_regional = uxgrid_global.subset.nearest_neighbor((0.0, 0.0), k=9)

(
    uxgrid_global.plot(
        periodic_elements="split",
        projection=ccrs.Robinson(),
        global_extent=True,
        features=["grid"],
        title="Global Grid",
    )
    + uxgrid_global_ocean.plot(
        periodic_elements="split",
        projection=ccrs.Robinson(),
        global_extent=True,
        features=["grid"],
        title="Global Grid with Holes",
    )
    + uxgrid_regional.plot(
        periodic_elements="split",
        projection=ccrs.Robinson(),
        global_extent=True,
        features=["grid"],
        title="Regional Grid",
    )
).cols(1)

### Point Data

In [ ]:
(
    uxgrid_global.plot.face_centers(
        projection=ccrs.Robinson(), global_extent=True, features=["grid"], s=2
    )
    + uxgrid_global_ocean.plot.face_centers(
        projection=ccrs.Robinson(), global_extent=True, features=["grid"], s=2
    )
    + uxgrid_regional.plot.face_centers(
        projection=ccrs.Robinson(), global_extent=True, features=["grid"], s=2
    )
).cols(1)

In [ ]:
x_global, y_global, z_global = (
    uxgrid_global.face_x.values,
    uxgrid_global.face_y.values,
    uxgrid_global.face_z.values,
)
points_global = (x_global, y_global, z_global)

In [ ]:
x_global_ocean, y_global_ocean, z_global_ocean = (
    uxgrid_global_ocean.face_x.values,
    uxgrid_global_ocean.face_y.values,
    uxgrid_global_ocean.face_z.values,
)
points_global_ocean = (x_global_ocean, y_global_ocean, z_global_ocean)

In [ ]:
x_regional, y_regional, z_regional = (
    uxgrid_regional.face_x.values,
    uxgrid_regional.face_y.values,
    uxgrid_regional.face_z.values,
)
points_regional = (x_regional, y_regional, z_regional)

## Global Data

###  Spherical Delaunay

In [ ]:
%%time
grid_dt = ux.Grid.from_points(points_global, method="spherical_delaunay")

In [ ]:
grid_dt.plot(
    projection=ccrs.Robinson(), line_width=0.5, title="Spherical Delaunay Triangles"
)

In [ ]:
(grid_dt.plot() * uxgrid_global.plot.face_centers(color="red")).opts(
    xlim=(-10, 10),
    ylim=(-5, 5),
    height=400,
    width=400,
    title="Spherical Delaunay Triangles (Zoomed)",
)

In [ ]:
grid_dt.n_max_face_nodes

In [ ]:
# todo, depending on the user's needs, data may need to be interpolated onto the face centers

### Spherical Voronoi

In [ ]:
%%time
grid_sv = ux.Grid.from_points(points_global, method="spherical_voronoi")

In [ ]:
grid_sv.plot(projection=ccrs.Robinson(), line_width=0.5)

In [ ]:
grid_sv.n_max_face_nodes

In [ ]:
(grid_sv.plot() * uxgrid_global.plot.face_centers(color="red")).opts(
    xlim=(-10, 10), ylim=(-5, 5), height=400, width=400
)

## Regional Data